# NIVA assignment 

By: Osman Gani

The Norwegian Structural Biology Center

Department of Chemisry

UiT-The Arctic University of Norway

Submitted: 13.05.2018

Please ignore the warning signs.

# 1. Data wrangling

Pandas dataframe is used to read the provided excel file.

In [1]:
# Import Pandas, the major Python library for this excercise
import pandas as pd

In [2]:
#Worksheet at Google drive
df = pd.read_excel('C:/temp/NIVA_assignment/tidal_sample.xlsx', 
                           parse_dates={'Timestamp': ['Date','Time']}, decimal=',' )

In [3]:
# See the data visually
df.head()


,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%)
0,03.04.2017 14:06,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10
1,03.04.2017 14:23,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0
2,11.04.2017 15:25,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1
3,11.04.2017 15:37,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0
4,11.04.2017 15:55,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0


In [4]:
df.shape

(250, 7)

### Convert date and time to Pandas Datetime series

In [5]:
df['Timestamp'] =  pd.to_datetime(df['Timestamp'],  errors='coerce')

In [6]:
df.shape

(250, 7)

One row is dropped because of the presence of invalid data in a row.

In [7]:
df_clean= df.dropna()

In [8]:
df_clean.shape

(249, 7)

### Make the timestep to ISO8601 format

Because the Kvratkert API can read the ISO8601 format.

Few more columns are temporarily added to use it for API access. These will be deleted later.

In [9]:
import datetime
df_clean['Time(ISO8601)'] = df_clean.Timestamp.map(lambda x: datetime.datetime.strftime(x, '%Y-%m-%dT%H:%M'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
df_clean.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Time(ISO8601)
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,2017-03-04T14:06
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2017-03-04T14:23
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,2017-11-04T15:25
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,2017-11-04T15:37
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,2017-11-04T15:55


In [12]:
df_clean['Nextdate'] = pd.to_datetime(df_clean['Timestamp']).apply(pd.DateOffset(minutes=10))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_clean.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Time(ISO8601),Nextdate
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,2017-03-04T14:06,2017-03-04 14:16:00
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2017-03-04T14:23,2017-03-04 14:33:00
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,2017-11-04T15:25,2017-11-04 15:35:00
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,2017-11-04T15:37,2017-11-04 15:47:00
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,2017-11-04T15:55,2017-11-04 16:05:00


In [14]:
df_clean['Next_Time(ISO8601)'] = df_clean.Nextdate.map(lambda x: datetime.datetime.strftime(x, '%Y-%m-%dT%H:%M'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_clean.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Time(ISO8601),Nextdate,Next_Time(ISO8601)
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,2017-03-04T14:06,2017-03-04 14:16:00,2017-03-04T14:16
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2017-03-04T14:23,2017-03-04 14:33:00,2017-03-04T14:33
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,2017-11-04T15:25,2017-11-04 15:35:00,2017-11-04T15:35
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,2017-11-04T15:37,2017-11-04 15:47:00,2017-11-04T15:47
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,2017-11-04T15:55,2017-11-04 16:05:00,2017-11-04T16:05


#  API access of Kartverket to read Water Levels

In [16]:
# Convert DataFrame series data to Python list (not very efficient)
lat_series= list(df_clean['GPS Latitude'])
lon_series= list(df_clean['GPS Longitude'])
Start_time_series= list(df_clean['Time(ISO8601)'])
End_time_series= list(df_clean['Next_Time(ISO8601)'])

In [19]:
# Latitide, longitude, and Times are used by a for loop
# The retrieved XML file is handled by lxml library to iterate through the tree structure

from urllib.request import urlopen
from lxml import etree

start_time= [] #Start time
wat_lev= [] # Captures the water levels at the position and time

waterlevel_api ='http://api.sehavniva.no/tideapi.php?tide_request=locationdata'
for a, b,c,d in zip(lat_series, lon_series, Start_time_series, End_time_series):
    lat=a
    lon=b
    starttime=c
    endtime=d
    
    waterlevel_req= waterlevel_api+'&lat=' + str(lat) + '&lon='+ str(lon)+'&datatype=ALL&dst=2&refcode=CD'\
    +'&fromtime='+starttime+'&totime='+endtime+'&interval=10'
    
    
    

    response = urlopen(waterlevel_req)
    response_read = response.read().decode("utf-8")
    
    
    root = etree.fromstring(response_read).getroottree()
    
    
    for elem in root.iter('data'):
        if elem.attrib['type']=='prediction':
            
            child= elem.find('waterlevel')
            start_time.append(c)
            wat_lev.append(child.attrib['value'])
            

In [23]:
# Make the water level in meter (float datatype) 
wat_lev_met= [float(i)/100 for i in wat_lev]

In [24]:
# Make another dataframe for the water level out (recived from Kartverket API)
df_wat = pd.DataFrame(
    {'Time(ISO8601)': start_time,
     'Water_Level': wat_lev_met,
    })

In [25]:
df_wat.head()

,Time(ISO8601),Water_Level
0,2017-03-04T14:06,1.962
1,2017-03-04T14:23,2.021
2,2017-11-04T15:25,0.685
3,2017-11-04T15:37,0.637
4,2017-11-04T15:55,0.559


In [26]:
# Merge the two dataframe (the initial and the above one)
df_merged_wat= pd.merge(df_clean, df_wat, on='Time(ISO8601)', how='outer')

In [86]:
df_merged_wat.depth =df_merged_wat.depth.astype(float)
df_merged_wat.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Time(ISO8601),Nextdate,Next_Time(ISO8601),Water_Level,Depth(CD_corrected)
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,2017-03-04T14:06,2017-03-04 14:16:00,2017-03-04T14:16,1.962,29.338
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2017-03-04T14:23,2017-03-04 14:33:00,2017-03-04T14:33,2.021,34.879
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,2017-11-04T15:25,2017-11-04 15:35:00,2017-11-04T15:35,0.685,30.915
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,2017-11-04T15:37,2017-11-04 15:47:00,2017-11-04T15:47,0.637,24.663
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,2017-11-04T15:55,2017-11-04 16:05:00,2017-11-04T16:05,0.559,32.141


### Normalized depth is calculated and added to the DataFrame

In [95]:
df_merged_wat['depth_normalized']=df_merged_wat['depth'] - df_merged_wat['Water_Level']

In [96]:
df_merged_wat.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Time(ISO8601),Nextdate,Next_Time(ISO8601),Water_Level,depth_normalized
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,2017-03-04T14:06,2017-03-04 14:16:00,2017-03-04T14:16,1.962,29.338
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2017-03-04T14:23,2017-03-04 14:33:00,2017-03-04T14:33,2.021,34.879
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,2017-11-04T15:25,2017-11-04 15:35:00,2017-11-04T15:35,0.685,30.915
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,2017-11-04T15:37,2017-11-04 15:47:00,2017-11-04T15:47,0.637,24.663
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,2017-11-04T15:55,2017-11-04 16:05:00,2017-11-04T16:05,0.559,32.141


In [97]:
df_renewed= df_merged_wat.copy(deep=True)

In [98]:
# Delete the previously added temporary columns
columns = ['Time(ISO8601)', 'Nextdate', 'Next_Time(ISO8601)', ]
df_renewed.drop(columns, inplace=True, axis=1)


In [99]:
df_renewed.head()

,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Water_Level,depth_normalized
0,2017-03-04 14:06:00,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,1.962,29.338
1,2017-03-04 14:23:00,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2.021,34.879
2,2017-11-04 15:25:00,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,0.685,30.915
3,2017-11-04 15:37:00,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,0.637,24.663
4,2017-11-04 15:55:00,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,0.559,32.141


# Save the Results in an Excel sheet

In [166]:
writer = pd.ExcelWriter('NIVA_Results.xlsx') #Saves in current directory
df_renewed.to_excel(writer,'Sheet1')
writer.save()

# Save the Results into a SQLite database

In [100]:
from sqlalchemy import create_engine # database connection

In [102]:
# Initializes database with filename NIVA_Assignment_v1.db in current directory
disk_engine = create_engine('sqlite:///NIVA_Assignment_v1.db') 

In [103]:
# Export the Dataframe into the database
df_renewed.to_sql('water', disk_engine)

# Database Query

In [104]:
# We read the data into a Pandas Dataframe again
df_sqlite = pd.read_sql_query('SELECT * FROM water', disk_engine)

In [105]:
df_sqlite

,index,Timestamp,Substrattype,GPS Latitude,GPS Longitude,depth,Kommentar til observasjon (evt fremmede arter),Anslått stortaresubstrat (%),Water_Level,depth_normalized
0,0,2017-03-04 14:06:00.000000,Grus-Stein,63.0902,7.26505,31.3,Vegetasjonsfattig grus/småsteinbunn med innsla...,10,1.962,29.338
1,1,2017-03-04 14:23:00.000000,Sand,63.10097,7.29703,36.9,Vegetasjonsfattig sandbunn. Strømrenner i sand...,0,2.021,34.879
2,2,2017-11-04 15:25:00.000000,Grus,62.50683,5.9377,31.6,Grusbunn med kalkalger (lite rhodolittdannelse...,1,0.685,30.915
3,3,2017-11-04 15:37:00.000000,Skjellsand,62.51823,5.93911,25.3,Vegetasjonsløs skjellsand med fine strømstriper,0,0.637,24.663
4,4,2017-11-04 15:55:00.000000,Skjellsand,62.52615,5.922,32.7,Vegetasjonsløs skjellsand med strømstriper. In...,0,0.559,32.141
5,5,2017-12-04 12:05:00.000000,Stein,62.56414,5.88093,43.3,Steinbunn med småvokste rødalger.,100,2.185,41.115
6,6,2017-06-23 13:14:00.000000,Stein,62.35022,5.42584,37.4,"Steinbunn med noe kortvokste rødalger, ellers ...",90,0.974,36.426
7,7,2017-06-04 16:00:00.000000,Sand-Stein,63.47915,8.1177,31.3,Sandbunn med stein. Vegetasjonsløst,40,0.827,30.473
8,8,2017-06-04 15:43:00.000000,Fjell,63.48384,8.12161,33.1,Vegetasjonsløs fjellbunn. Innslag av svamp,100,0.784,32.316
9,9,2017-06-04 16:17:00.000000,Fjell-Stein,63.48167,8.07677,32.8,Vegetasjonsløs fjellbunn. Innslag av kortvokst...,100,0.919,31.881


In [110]:
average_depth = pd.read_sql_query('SELECT AVG(depth_normalized) FROM water', disk_engine)
minimum_depth = pd.read_sql_query('SELECT MIN(depth_normalized) FROM water', disk_engine)
max_depth = pd.read_sql_query('SELECT MAX(depth_normalized) FROM water', disk_engine)

In [109]:
average_depth

,AVG(depth_normalized)
0,18.696839


In [111]:
minimum_depth

,MIN(depth_normalized)
0,0.92


In [112]:
max_depth

,MAX(depth_normalized)
0,41.115


In [124]:
%matplotlib inline

# Showing sampling points on the map

In [155]:
# Convert the locations strings into float
lat_fl=[float(i) for i in lat_series]
lon_fl=[float(i) for i in lon_series]

In [163]:
import folium
mapit = folium.Map( location=[62.5, 6.5], zoom_start=7 )
for coord in zip(lat_fl, lon_fl):
    folium.Marker( location=[ coord[0], coord[1] ]).add_to( mapit )

mapit